In [1]:
#!pip install pybaseball

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 25.8 MB/s eta 0:00:00


https://github.com/jldbc/pybaseball

In [3]:
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading in data



In [40]:
## Grabbing Chris Sale and Tarik Skubal
pd.concat([playerid_lookup('sale', 'chris'), playerid_lookup('skubal', 'tarik')])

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,sale,chris,519242,salec001,salech01,10603,2010.0,2024.0
0,skubal,tarik,669373,skubt001,skubata01,22267,2020.0,2024.0


In [46]:
## Putting their pitch by pitch data into a data frame
sale_2024 = statcast_pitcher('2024-03-28', '2024-11-02', 519242)
skubal_2024 = statcast_pitcher('2024-03-28', '2024-11-02', 669373)

Gathering Player Data
Gathering Player Data


The columns go from most recent to least recent. I want this order to be switched, since I am looking to sequence from beginning to end, not vise versa.

In [47]:
sale_2024 = sale_2024.iloc[::-1].reset_index(drop=True)
skubal_2024 = skubal_2024.iloc[::-1].reset_index(drop=True)

# Column Cleaning

Many of these columns are not important, let's explore this deeper. I will drop all columns not necessary for this project.

In [48]:
print(sale_2024.columns.tolist())


['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_a

In [49]:
columns_to_keep = [
    'pitch_type', 'release_speed', 'release_pos_x', 'release_pos_y', 'release_pos_z',
    'release_spin_rate', 'spin_axis', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'sz_top', 'sz_bot',
    'balls', 'strikes', 'outs_when_up', 'inning', 'inning_topbot', 'stand', 'p_throws',
    'bat_score', 'fld_score', 'at_bat_number', 'pitch_number', 'game_pk', 'batter', 'pitcher',
    'description', 'events'
]

sale_2024 = sale_2024[columns_to_keep]
skubal_2024 = skubal_2024[columns_to_keep]

# Why did I drop all other columns?

* **fielder_2 ... fielder_9**: Do not need to know who the others on the field are
* **if_fielding_alignment, of_fielding_alignment:** Is not related to pitch sequencing
* **hit_location, hit_distance_sc, launch_speed, launch_angle, launch_speed_angle, estimated_ba_using_speedangle, estimated_woba_using_speedangle, woba_value, woba_denom, babip_value, iso_value, estimated_slg_using_speedangle, post_away_score, post_home_score, post_bat_score, post_fld_score, delta_home_win_exp, delta_run_exp, delta_pitcher_run_exp, home_score_diff, bat_score_diff, home_win_exp, bat_win_exp:** These are all about what happens after the pitch was thrown. This could bring risk of data leakage, because it is future information that an RNN should not have access to. They also do not have anything to do with the actual pitch sequence.
* **tfs_deprecated, tfs_zulu_deprecated, umpire, game_year, hc_x, hc_y:** These metrics are redundant, as they either add value that doesn't matter or they add value that is already found elsewhere in the data.
* **age_bat_legacy, age_pit_legacy, age_bat, n_priorpa_thisgame_player_at_bat, batter_days_since_prev_game, pitcher_days_until_next_game, batter_days_until_next_game:** This is more batter and pitcher historical data. They describe historical (or future) performance rather than real-time sequencing in games. While this is sometimes important in order to understand performance, they are noise when it comes to using RNNs.

# Why did I keep these columns?

The columns I kept define the game state, pitch characteristics, and context necessary for sequencing. For example, pitch_type, release_speed, and the other pitcher metrics describe what the pitcher does (and really who they are as a pitcher). Game context (like balls, strikes, and inning) are important because they affect decision making. The rest of the columns kept are necessary to sequence the data.



# Missing Data

Now that the columns are the ones we want, I am going to explore if there are missing data in these columns, and what to do about it.

In [52]:
pd.DataFrame({
    'sale_2024_missing': sale_2024.isna().sum(),
    'skubal_2024_missing': skubal_2024.isna().sum()
})

,sale_2024_missing,skubal_2024_missing
pitch_type,0,28
release_speed,0,28
release_pos_x,0,28
release_pos_y,0,28
release_pos_z,0,28
release_spin_rate,0,55
spin_axis,0,55
pfx_x,0,28
pfx_z,0,28
plate_x,0,28


This is interesting. The only column that sale has missing data in is "events", while Skubal has some in many other columns. The tricky part about dealing with missing data in this case is that it is sequenced. That is, I can't just remove rows with missing data because that messes up the sequence of it all.

With "events", NA makes sense, as the event column is only filled when the at-bat is up. I will impute these rows with "batter still up" as a place holder. After this is done, I will do some more deep exploring into what the reason is behind these missing values.

In [53]:
## Imputing missing values in the 'events' column with 'batter still up'
sale_2024['events'] = sale_2024['events'].fillna('batter still up')
skubal_2024['events'] = skubal_2024['events'].fillna('batter still up')

Let's look into Skubal's rows with missing data.

In [54]:
skubal_2024[skubal_2024.isna().any(axis=1)]

,pitch_type,release_speed,release_pos_x,release_pos_y,release_pos_z,release_spin_rate,spin_axis,pfx_x,pfx_z,plate_x,plate_z,sz_top,sz_bot,balls,strikes,outs_when_up,inning,inning_topbot,stand,p_throws,bat_score,fld_score,at_bat_number,pitch_number,game_pk,batter,pitcher,description,events
0,FF,95.7,1.64,54.13,6.47,NaN,NaN,0.71,1.47,-0.85,3.61,3.31,1.54,0,0,0,1,Bot,L,L,0,0,4,1,746821,643217,669373,ball,batter still up
1,SI,94.5,1.95,54.13,6.30,NaN,NaN,1.41,1.05,0.50,2.61,3.27,1.51,1,0,0,1,Bot,L,L,0,0,4,2,746821,643217,669373,foul,batter still up
2,SL,86.4,1.89,54.13,6.38,NaN,NaN,-0.12,0.48,-0.99,2.63,3.35,1.53,1,1,0,1,Bot,L,L,0,0,4,3,746821,643217,669373,ball,batter still up
3,FF,96.1,1.81,54.14,6.36,NaN,NaN,0.54,1.46,-0.43,2.47,3.27,1.51,2,1,0,1,Bot,L,L,0,0,4,4,746821,643217,669373,swinging_strike,batter still up
4,FF,98.0,1.81,54.13,6.33,NaN,NaN,0.81,1.38,0.33,3.01,3.27,1.51,2,2,0,1,Bot,L,L,0,0,4,5,746821,643217,669373,foul,batter still up
5,SI,96.4,1.93,54.13,6.24,NaN,NaN,1.41,1.14,0.47,3.97,3.27,1.51,2,2,0,1,Bot,L,L,0,0,4,6,746821,643217,669373,foul,batter still up
6,SL,87.6,2.10,54.13,6.26,NaN,NaN,-0.26,0.35,-1.05,1.20,3.27,1.51,2,2,0,1,Bot,L,L,0,0,4,7,746821,643217,669373,swinging_strike,strikeout
7,FF,97.4,1.75,54.13,6.40,NaN,NaN,0.50,1.26,0.08,3.53,3.31,1.47,0,0,1,1,Bot,R,L,0,0,5,1,746821,660162,669373,hit_into_play,field_out
8,CH,86.0,1.77,54.13,6.32,NaN,NaN,1.19,0.68,-0.38,2.81,3.78,1.78,0,0,2,1,Bot,R,L,0,0,6,1,746821,673357,669373,called_strike,batter still up
9,CH,86.0,1.61,54.13,6.36,NaN,NaN,1.28,0.36,-0.12,2.04,3.74,1.77,0,1,2,1,Bot,R,L,0,0,6,2,746821,673357,669373,hit_into_play,single


I am going to approach this in two ways: for the rows that are missing pitch type, I will just delete the sequence that it is in. For the rows that have pitch type, but are missing the other columns, I will impute the missing values with the average of that metric based on the given pitch. Release rate and spin axis change based on the type of pitch, which is why I think it is appropriate to approach it like this.

In [60]:
def find_missing_pitch_type_combinations(df, at_bat_col='at_bat_number', game_pk_col='game_pk', pitch_type_col='pitch_type'):

    nan_pitch_type_rows = df[df[pitch_type_col].isna()]
    return nan_pitch_type_rows[[at_bat_col, game_pk_col]].drop_duplicates()


missing_combinations = find_missing_pitch_type_combinations(skubal_2024)
missing_combinations

,at_bat_number,game_pk
2202,1,746431
2210,2,746431
2213,3,746431
2216,7,746431
2218,8,746431
2224,9,746431


Now that I know that these are the at_bat_number and game_pk combinations that have missing. I need to remove these entire at-bats to make sure that the sequences are still in tact. To do so, I will loop through this data frame and delete combinations in the actual skubal data frame.

In [61]:
for index, row in missing_combinations.iterrows():
    at_bat = row['at_bat_number']
    game_pk = row['game_pk']

    ## Creating a boolean mask for rows to delete
    mask = (skubal_2024['at_bat_number'] == at_bat) & (skubal_2024['game_pk'] == game_pk)

    ## Deleting the matching rows
    skubal_2024 = skubal_2024[~mask]


Let's check to see if this worked.

In [62]:
skubal_2024[skubal_2024.isna().any(axis=1)]

,pitch_type,release_speed,release_pos_x,release_pos_y,release_pos_z,release_spin_rate,spin_axis,pfx_x,pfx_z,plate_x,plate_z,sz_top,sz_bot,balls,strikes,outs_when_up,inning,inning_topbot,stand,p_throws,bat_score,fld_score,at_bat_number,pitch_number,game_pk,batter,pitcher,description,events
0,FF,95.7,1.64,54.13,6.47,NaN,NaN,0.71,1.47,-0.85,3.61,3.31,1.54,0,0,0,1,Bot,L,L,0,0,4,1,746821,643217,669373,ball,batter still up
1,SI,94.5,1.95,54.13,6.30,NaN,NaN,1.41,1.05,0.50,2.61,3.27,1.51,1,0,0,1,Bot,L,L,0,0,4,2,746821,643217,669373,foul,batter still up
2,SL,86.4,1.89,54.13,6.38,NaN,NaN,-0.12,0.48,-0.99,2.63,3.35,1.53,1,1,0,1,Bot,L,L,0,0,4,3,746821,643217,669373,ball,batter still up
3,FF,96.1,1.81,54.14,6.36,NaN,NaN,0.54,1.46,-0.43,2.47,3.27,1.51,2,1,0,1,Bot,L,L,0,0,4,4,746821,643217,669373,swinging_strike,batter still up
4,FF,98.0,1.81,54.13,6.33,NaN,NaN,0.81,1.38,0.33,3.01,3.27,1.51,2,2,0,1,Bot,L,L,0,0,4,5,746821,643217,669373,foul,batter still up
5,SI,96.4,1.93,54.13,6.24,NaN,NaN,1.41,1.14,0.47,3.97,3.27,1.51,2,2,0,1,Bot,L,L,0,0,4,6,746821,643217,669373,foul,batter still up
6,SL,87.6,2.10,54.13,6.26,NaN,NaN,-0.26,0.35,-1.05,1.20,3.27,1.51,2,2,0,1,Bot,L,L,0,0,4,7,746821,643217,669373,swinging_strike,strikeout
7,FF,97.4,1.75,54.13,6.40,NaN,NaN,0.50,1.26,0.08,3.53,3.31,1.47,0,0,1,1,Bot,R,L,0,0,5,1,746821,660162,669373,hit_into_play,field_out
8,CH,86.0,1.77,54.13,6.32,NaN,NaN,1.19,0.68,-0.38,2.81,3.78,1.78,0,0,2,1,Bot,R,L,0,0,6,1,746821,673357,669373,called_strike,batter still up
9,CH,86.0,1.61,54.13,6.36,NaN,NaN,1.28,0.36,-0.12,2.04,3.74,1.77,0,1,2,1,Bot,R,L,0,0,6,2,746821,673357,669373,hit_into_play,single


Perfect. Now that the only missing values are in the release_spin_rate and spin_axis, I can impute these based on the averages from that spin type.

In [63]:
# Impute 'release_spin_rate' and 'spin_axis' based on the mean of their respective 'pitch_type' group
skubal_2024['release_spin_rate'] = skubal_2024.groupby('pitch_type')['release_spin_rate'].transform(lambda x: x.fillna(x.mean()))
skubal_2024['spin_axis'] = skubal_2024.groupby('pitch_type')['spin_axis'].transform(lambda x: x.fillna(x.mean()))

Let's recheck missing data.

In [64]:
pd.DataFrame({
    'sale_2024_missing': sale_2024.isna().sum(),
    'skubal_2024_missing': skubal_2024.isna().sum()
})

,sale_2024_missing,skubal_2024_missing
pitch_type,0,0
release_speed,0,0
release_pos_x,0,0
release_pos_y,0,0
release_pos_z,0,0
release_spin_rate,0,0
spin_axis,0,0
pfx_x,0,0
pfx_z,0,0
plate_x,0,0


Now want to save this data so that I don't have to rerun this script every time.

In [65]:
sale_2024.to_csv('sale_2024_cleaned.csv', index=False)
skubal_2024.to_csv('skubal_2024_cleaned.csv', index=False)

# Next Steps

The data is now "clean". Outliers are still there, the data is not yet scaled, and the categorical data is not one-hot encoded yet. However, I want to do the EDA before I actually do this type of cleaning.

Here are the next steps:



1.   Exploratory Data Analysis
2.   Further Data Cleaning (Outliers, One_hot_encoding, scaling)
3. Feature Engineering/PCA

